In [1]:
import pandas as pd
import sys
import glob
import datetime
from pathlib import Path
!{sys.executable} -m pip install xlrd

In [2]:
def create_dataframe(file):
    """
    Create a DataFrame from the data on the given source.

    Parameters
    ----------
        file : Path to the file where the data is.

    Returns
    -------
        data: DataFrame for the given stock

    """
    
    data = pd.read_csv(file)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date')
    return data

In [3]:
def to_datetime(date):
    return datetime.datetime.strptime(date, '%Y-%m-%d')

def period_end_date(n_days, date):
    a_date = to_datetime(date)
    days = datetime.timedelta(n_days)
    return (a_date + days).strftime('%Y-%m-%d')

In [4]:
def get_mask(data, start, period):
    """
    Gets the mask from a given startig date and a calculated end date (based on the start and period)

    Parameters
    ----------
        start : Starting date
        period : The number of days that want the mask to comprise

    Returns
    -------
        mask: The mask between start and start+period

    """
    
    end = period_end_date(period, start)
    return (data['Date'] > start) & (data['Date'] <= end)

In [5]:
def comission_formula(total, current_stock_val, stock_threshold, fixed_commission, dynamic_commission):
    """
    Perform rebalance of stock based on given parameters

    Parameters
    ----------
        total : The total amount available for the stock
        current_stock_val : The amount that is already allocated to the stock
        stock_threshold : part of the available cash the will be in the stock, values between 0 and 1
        fixed_commission : fixed commission paid on a given transaction
        dynamic_commission : percentage of the money in the transaction that will be charged as commission

    Returns
    -------
        stock_val: New stock portfolio value after transaction
        cash_val: Final value when rebalance is applied
        commission_val: Amount paid as commission for this transaction

    """
    
    # If current stock value is higher than future value
    val1 = (total-fixed_commission+current_stock_val*dynamic_commission)/((1/stock_threshold) + dynamic_commission)
    # If current stock value is lower than future value
    val2 = (total-fixed_commission-current_stock_val*dynamic_commission)/((1/stock_threshold) - dynamic_commission)
    
    stock_val = val1 if (val1-current_stock_val) > 0 else val2
    commission_val = fixed_commission + dynamic_commission*abs(stock_val - current_stock_val)
    cash_val = total - stock_val - commission_val
    return stock_val, cash_val, commission_val

def rebalance(initial_cash, stock_threshold, up_trade, low_trade, fixed_commission, dynamic_commission, prices):
    """
    Perform rebalance of stock based on given parameters

    Parameters
    ----------
        initial_cash : The initial amount available for the stock
        stock_threshold : part of the available cash the will be in the stock, values between 0 and 1
        up_trade : upward momentum needed between adjusted stock and cash values to make the transacrion
        low_trade : downward momentum needed between adjusted stock and cash values to make the transacrion
        fixed_commission : fixed commission paid on a given transaction
        dynamic_commission : percentage of the money in the transaction that will be charged as commission
        prices : dataframe containing the prices for a given period

    Returns
    -------
        long: The value that would be obtained if we took a long position on a stock
        total: Final value when rebalance is applied
        count: Number of transactions executed

    """
    
    cash_threshold = 1 - stock_threshold
    
    total = initial_cash
    stock_val, cash_val, commission_val = comission_formula(total, 0, stock_threshold, fixed_commission, dynamic_commission)
    
    long_shares = initial_cash if prices.empty else initial_cash/prices.iloc[0]
    rebalance_shares = initial_cash if prices.empty else stock_val/prices.iloc[0]
    
    count = 0 if prices.empty else 1
    price = 1
    
    for price in prices:
        stock_val = price * rebalance_shares
        total = stock_val + cash_val
        
        # Check how far apart are the balances from rebalanced portfolio
        if (stock_val*cash_threshold)/(cash_val*stock_threshold) > 1+up_trade or \
        (stock_val*cash_threshold)/(cash_val*stock_threshold) < 1-low_trade:
            stock_val, cash_val, commission_val = comission_formula(
                total, stock_val, stock_threshold, fixed_commission, dynamic_commission)
            rebalance_shares = stock_val/price
            count += 1
    
    return long_shares*price, total, count

In [6]:
def rebalance_result(file, start, period, investment, stock_threshold, up_trade,
                     low_trade, fixed_commission, dynamic_commission):
    """
    Perform rebalance of stock based on given parameters

    Parameters
    ----------
        file : file containing the data
        start : Starting date of your analysis
        period : The offset number of days that want to comprise
        investment : The initial amount available for the stock
        stock_threshold : part of the available cash the will be in the stock, values between 0 and 1
        up_trade : upward momentum needed between adjusted stock and cash values to make the transacrion
        low_trade : downward momentum needed between adjusted stock and cash values to make the transacrion
        fixed_commission : fixed commission paid on a given transaction
        dynamic_commission : percentage of the money in the transaction that will be charged as commission
        prices : dataframe containing the prices for a given period

    Returns
    -------
        long: The value that would be obtained if we took a long position on a stock
        total: Final value when rebalance is applied
        count: Number of transactions executed

    """
    data = create_dataframe(file)
    filtered_data = data.loc[get_mask(data, start, period)]
    close = filtered_data['Adj Close']
    return rebalance(investment, stock_threshold, up_trade, low_trade, fixed_commission, dynamic_commission, close)
    

In [7]:
def print_result(investment, l_val, r_val, c):
    l_change = (l_val/investment-1)*100
    r_change = (r_val/investment-1)*100
    print("Investment: %5.2f$, \nLong value: %5.2f$ change: %5.2f%%, \nRebalance value: %5.2f$ change: %5.2f%%, \nNum transactions: %d" % 
          (investment, l_val, l_change, r_val, r_change, c))

In [8]:
# Parameters to determine the period to analyse
start = '2015-01-01'
period = 2000

# Investment parameters
investment = 200
stock_threshold = 0.5
up_trade = 0.0
low_trade = 0.0
fixed_commission = 0.0
dynamic_commission = 0

In [9]:
"""
Calculate the final values of the rebalanced and long position for a single stock.
"""
paths = glob.glob("**/AAPL.csv", recursive=True)
if paths:
    stock_file = paths[0]
    print(Path(stock_file).stem)
    l_val, r_val, c = rebalance_result(stock_file, start, period, investment, stock_threshold, 
                             up_trade, low_trade, fixed_commission, dynamic_commission)
    print_result(investment, l_val, r_val, c)
else:
    print("No data for this stock")

AAPL
Investment: 200.00$, 
Long value: 521.66$ change: 160.83%, 
Rebalance value: 340.17$ change: 70.09%, 
Num transactions: 1257


In [10]:
total = 0
port_long = 0
port_rebal = 0
port_transactions = 0

"""
Loop through the data files and calculate the final values of the rebalanced portfolio and 
the long position portfolio.
"""
for file in glob.glob("Portfolio/*.csv", recursive=True):
    l_val, r_val, c = rebalance_result(file, start, period, investment, stock_threshold, 
                             up_trade, low_trade, fixed_commission, dynamic_commission)
    total += investment
    port_long += l_val
    port_rebal += r_val
    port_transactions += c

print_result(total, port_long, port_rebal, port_transactions)

Investment: 5200.00$, 
Long value: 15024.68$ change: 188.94%, 
Rebalance value: 8107.09$ change: 55.91%, 
Num transactions: 29743


In [11]:
"""
Loop through the data files and calculate the final values of the rebalanced and 
the long position in the main indexes.
"""
for file in glob.glob("Index/*.csv", recursive=True):
    print(Path(file).stem)
    l_val, r_val, c = rebalance_result(file, start, period, investment, stock_threshold, 
                             up_trade, low_trade, fixed_commission, dynamic_commission)
    print_result(investment, l_val, r_val, c)

RUSSELL2000
Investment: 200.00$, 
Long value: 198.97$ change: -0.51%, 
Rebalance value: 206.21$ change:  3.10%, 
Num transactions: 1258
NASDAQ
Investment: 200.00$, 
Long value: 354.33$ change: 77.16%, 
Rebalance value: 273.63$ change: 36.81%, 
Num transactions: 1259
DJIA
Investment: 200.00$, 
Long value: 258.59$ change: 29.29%, 
Rebalance value: 233.03$ change: 16.51%, 
Num transactions: 1258
S&P500
Investment: 200.00$, 
Long value: 268.76$ change: 34.38%, 
Rebalance value: 237.16$ change: 18.58%, 
Num transactions: 1258
